<a href="https://colab.research.google.com/github/Ruturajjadhav35/IR_search_engine/blob/main/Search_engine_for_information_retrevial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nechbamohammed/research-papers-dataset")

print("Path to dataset files:", path)

100%|██████████| 590M/590M [00:09<00:00, 64.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/nechbamohammed/research-papers-dataset/versions/1


In [ ]:
csv_path = '/root/.cache/kagglehub/datasets/nechbamohammed/research-papers-dataset/versions/1/dblp-v10.csv'

In [ ]:
import pandas as pd
df = pd.read_csv(csv_path)
df = df.dropna()
# Optional: reset the index
df = df.reset_index(drop=True)
df.columns


Index(['abstract', 'authors', 'n_citation', 'references', 'title', 'venue',
       'year', 'id'],
      dtype='object')

In [ ]:
df = df[['abstract', 'title', 'venue', 'year', 'authors']]
df = df.sample(n=1000, random_state=42).reset_index(drop=True)




In [ ]:
top_25_years = df['authors'].value_counts().head(25)
print("Top 25 most frequent authors:")
print(top_25_years)

Top 25 most frequent authors:
authors
['Valery Sklyarov', 'Iouliia Skliarova']                                                    1
['Abdulhadi Shoufan', 'Sorin A. Huss', 'Oliver Kelm', 'Sebastian Schipp']                   1
['Kevin Zhu']                                                                               1
['Caitlin Moore', 'Lyn Robinson']                                                           1
['Ilan Kremer', 'Andrzej Skrzypacz']                                                        1
["Melwyn D'Souza", 'V. S. Ananthanarayana']                                                 1
['Joanne M. Wood']                                                                          1
['Sylvain Schmitz']                                                                         1
['Andres Larrea', 'Oscar Barambones', 'Jose A. Ramos-Herran']                               1
['Marc C. Necker', 'Andreas Weber']                                                         1
['Jeffrey Wildman', 'S

In [ ]:
top_25_years = df['year'].value_counts().head(25)
print("Top 25 most frequent years:")
print(top_25_years)

Top 25 most frequent years:
year
2016    182
2008     77
2009     69
2015     63
2006     59
2007     56
2012     51
2011     47
2010     45
2005     38
2017     37
2004     37
2014     34
2013     29
2002     29
2003     25
2000     21
2001     16
1998     11
1993      9
1994      9
1997      9
1996      8
1991      7
1995      6
Name: count, dtype: int64


In [ ]:
top_25_titles = df['title'].value_counts().head(25)
print("Top 25 most frequent titles:")
print(top_25_titles)

Top 25 most frequent titles:
title
Teaching reconfigurable systems: methods, tools, tutorials, and projects                                                                             1
A Novel Rekeying Message Authentication Procedure Based on Winternitz OTS and Reconfigurable Hardware Architectures                                  1
The Complementarity of Information Technology Infrastructure and E-Commerce Capability: A Resource-Based Assessment of Their Business Value          1
The Monty Hyams Archive: A new resource for the information history of the late twentieth century                                                    1
Dynamic signaling and market breakdown                                                                                                               1
Enhanced lbs discovery in a decentralized registry based web services environment                                                                    1
AGE AND VISUAL IMPAIRMENT DECREASE DRIVING PERFORMANCE AS M

In [ ]:
top_25_venues = df['venue'].value_counts().head(25)
print("Top 25 most frequent venues:")
print(top_25_venues)

Top 25 most frequent venues:
venue
international conference on acoustics, speech, and signal processing                13
international conference on communications                                          13
international geoscience and remote sensing symposium                               12
global communications conference                                                    12
international conference on image processing                                        12
winter simulation conference                                                        11
international conference on robotics and automation                                 11
IEEE Transactions on Signal Processing                                              10
Bioinformatics                                                                      10
vehicular technology conference                                                      9
intelligent robots and systems                                                       8
IEEE Tra

In [ ]:
  # Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from numpy.linalg import svd
from sklearn.metrics.pairwise import cosine_similarity


# Step 1: Preprocessing and Feature Extraction
def preprocess_text(text):
    # Simple preprocessing for demonstration: lowercasing and removing punctuation
    return text.lower().replace('[^\w\s]', '')

# Combine the columns into a single content column
df['content'] = df.apply(
    lambda x: f"{x['title']} {x['abstract']} {' '.join(x['authors'])} {x['venue']} {str(x['year'])}",
    axis=1
)

# Applying preprocessing
df['cleaned_content'] = df['content'].apply(preprocess_text)

# Step 2: Constructing the TF-IDF matrix with Bi-grams
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
tfidf_matrix = vectorizer.fit_transform(df['cleaned_content']).toarray()

# Step 3: Applying SVD
U, S, VT = svd(tfidf_matrix, full_matrices=False)  # Use economy SVD as it is computationally efficient

# Normalizing the left singular vectors (document-topic matrix)
normalizer = Normalizer()
U_normalized = normalizer.fit_transform(U * S)  # Element-wise multiplication for projection


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from numpy.linalg import svd
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Preprocessing and Feature Extraction
def preprocess_text(text):
    # Simple preprocessing: lowercasing and removing punctuation
    return text.lower().replace('[^\w\s]', '')

# Combine the columns into a single content column
df['content'] = df.apply(
    lambda x: f"{x['title']} {x['abstract']} {' '.join(x['authors'])} {x['venue']} {str(x['year'])}",
    axis=1
)

# Applying preprocessing
df['cleaned_content'] = df['content'].apply(preprocess_text)

# Step 2: Constructing the Term Frequency matrix with Bi-grams
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 3))
tf_matrix = vectorizer.fit_transform(df['cleaned_content']).toarray()

# Step 3: Applying SVD
U, S, VT = svd(tf_matrix, full_matrices=False)  # Economy SVD for efficiency

# Normalizing the left singular vectors (document-topic matrix)
normalizer = Normalizer()
U_normalized = normalizer.fit_transform(U * S)  # Element-wise multiplication for projection


In [ ]:
query = "I would convolutional neural networks"
query_vector = vectorizer.transform([query]).toarray()

# Project the query vector into the same SVD space
query_svd = query_vector.dot(VT.T)

# Normalizing the query SVD vector
query_svd_normalized = normalizer.transform(query_svd)

# Document Scoring and Ranking
cosine_similarities = cosine_similarity(query_svd_normalized, U_normalized)

# Retrieving the indices of the top 5 most similar documents
top_indices = np.argsort(cosine_similarities.flatten())[-1:][::-1]

# Output the top 5 results
print("Top 5 most similar documents by SVD and TF-IDF with Bi-grams:")
for index in top_indices:
    print(f"Title: {df.iloc[index]['title']}")
    print(f"Abstract: {df.iloc[index]['abstract']}")
    print(f"Venue: {df.iloc[index]['venue']}")
    print(f"Year: {df.iloc[index]['year']}")
    print(f"Authors: {df.iloc[index]['authors']}")
    print("-" * 80)


Top 5 most similar documents by SVD and TF-IDF with Bi-grams:
Title: New Delay-Dependent Stability Results for Neural Networks With Time-Varying Delay
Abstract: This paper studies the problem of stability analysis for neural networks (NNs) with a time-varying delay. Unlike the previous works, the activation functions are assumed to be neither monotonic, nor differentiable, nor bounded. By defining a more general type of Lyapunov functionals, some new less conservative delay-dependent stability criteria are established in terms of linear matrix inequalities (LMIs). Meanwhile, the computational complexity of the newly obtained stability conditions is reduced because less variables are involved. Numerical examples are given to illustrate the effectiveness and the benefits of the proposed method.
Venue: IEEE Transactions on Neural Networks
Year: 2008
Authors: ['Xun-Lin Zhu', 'Guang-Hong Yang']
--------------------------------------------------------------------------------


In [ ]:
#venue test
import re
import numpy as np

# Function to extract the expected venue from a query using regex.
def extract_expected_venue(query):
    """
    Extracts the expected venue from the query string by searching for the keywords "from" or "in".
    Once extracted, it converts the venue to lowercase and removes occurrences of the word 'the'.
    Returns the cleaned venue string or None if no match is found.
    """
    match = re.search(r'(?:from|in)\s+(.*?)[\.,\?]?\s*$', query, re.IGNORECASE)
    if match:
        # Extract, lowercase, and remove the whole word "the"
        venue = match.group(1).strip().lower()
        venue = re.sub(r'\bthe\b', '', venue).strip()
        # Remove extra spaces if any
        venue = ' '.join(venue.split())
        return venue
    else:
        return None

# List of sample venue queries for evaluation.
sample_queries = [
    "I would like research papers from the International Conference on Acoustics, Speech, and Signal Processing.",
    "I would like research papers from the International Conference on Communications.",
    "I would like research papers from the Global Communications Conference.",
    "I would like research papers from the International Conference on Image Processing.",
    "I would like research papers from the International Conference on Robotics and Automation.",

    "Can you show me publications from Intelligent Robots and Systems?",
    "Can you show me publications from the Winter Simulation Conference?",
    "Can you show me publications from IEEE Transactions on Signal Processing?",
    "Can you show me publications from the Vehicular Technology Conference?",
    "Can you show me publications from Bioinformatics?",

    " papers published in the International Geoscience and Remote Sensing Symposium.",
    " papers published in IEEE Transactions on Information Theory.",
    "papers published in Human Factors in Computing Systems.",
    " papers published in the International conference of IEEE engineering in medicine and biology society.",
    "papers published in the Conference on Decision and Control.",

    "Could you retrieve studies from the National Conference on Artificial Intelligence?",
    "Could you retrieve studies from IEEE Transactions on Biomedical Engineering?",
    "Could you retrieve studies from the Hawaii International Conference on System Sciences?",
    "Could you retrieve studies from IEEE Transactions on Communications?",
    "Could you retrieve studies from the International Symposium on Circuits and Systems?",

    "I'm looking for academic papers from the Wireless Communications and Networking Conference.",
    "I'm looking for academic papers from the IEEE Journal on Selected Areas in Communications.",
    "I'm looking for academic papers from Sensors.",
    "I'm looking for academic papers from the IEEE International Conference on Fuzzy Systems.",
    "I'm looking for academic papers from Neural Information Processing Systems."
]

# Initialize a list to store precision for each query.
precision_scores = []

# Loop over each query to perform retrieval, display venues, and calculate evaluation metrics.
for query in sample_queries:
    expected_venue = extract_expected_venue(query)
    print("Query:", query)
    print("Expected Venue:", expected_venue)

    # Project the query into the SVD space using the previously defined vectorizer, VT, and normalizer.
    query_vector = vectorizer.transform([query]).toarray()
    query_svd = query_vector.dot(VT.T)
    query_svd_normalized = normalizer.transform(query_svd)

    # Compute cosine similarities between the query SVD and all normalized document vectors.
    cosine_similarities = cosine_similarity(query_svd_normalized, U_normalized)

    # Retrieve the indices of the top 5 most similar documents.
    top_indices = np.argsort(cosine_similarities.flatten())[-5:][::-1]

    # Print out the cleaned venues of the top 5 documents.
    print("Top 5 Document Venues:")
    for idx in top_indices:
        doc_venue = df.iloc[idx]['venue'].lower().strip()
        doc_venue = re.sub(r'\bthe\b', '', doc_venue).strip()
        doc_venue = ' '.join(doc_venue.split())
        print(doc_venue)

    # Count the number of retrieved documents where the document venue matches the expected venue.
    TP = 0
    for idx in top_indices:
        doc_venue = df.iloc[idx]['venue'].lower().strip()
        doc_venue = re.sub(r'\bthe\b', '', doc_venue).strip()
        doc_venue = ' '.join(doc_venue.split())
        if doc_venue == expected_venue:
            TP += 1

    # Calculate precision based on the 5 retrieved documents.
    precision = TP / 5.0
    precision_scores.append(precision)

    print("Precision: {:.2f}".format(precision))
    print("-" * 80)

# Calculate average precision over all queries.
average_precision = np.mean(precision_scores)
print("Average Precision over all queries: {:.2f}".format(average_precision))


Query: I would like research papers from the International Conference on Acoustics, Speech, and Signal Processing.
Expected Venue: international conference on acoustics, speech, and signal processing
Top 5 Document Venues:
international conference on acoustics, speech, and signal processing
international conference on acoustics, speech, and signal processing
international conference on acoustics, speech, and signal processing
international conference on acoustics, speech, and signal processing
international conference on acoustics speech and signal processing
Precision: 0.80
--------------------------------------------------------------------------------
Query: I would like research papers from the International Conference on Communications.
Expected Venue: international conference on communications
Top 5 Document Venues:
international conference on communications
international conference on communications
international conference on communications
research and practice in technology e

In [ ]:
import re
import numpy as np

# Function to extract the expected year from a query using regex.
def extract_expected_year(query):
    """
    Extracts the expected year (4-digit) from the query string.
    Returns the year as a string or None if not found.
    """
    match = re.search(r'\b(19|20)\d{2}\b', query)
    if match:
        return match.group(0)
    else:
        return None

# List of sample year-based queries with varied wording.
sample_queries = [
    "I'm looking for research papers from 2016.",
    "Can you show me studies published in 2008?",
    "I’d like to see academic publications from 2009.",
    "Do you have any papers from 2015?",
    "Please find research papers that came out in 2006.",
    "Could you pull up studies from 2007?",
    "I want to browse papers from 2012.",
    "Can I get access to research from 2011?",
    "Show me academic articles published in 2010.",
    "I'd like to check out studies from the year 2005.",
    "Retrieve publications from 2017, please.",
    "Give me research papers published around 2004.",
    "I’m interested in work from the year 2014.",
    "Could you list papers from 2013?",
    "I’m searching for articles from 2002.",
    "Please show me research published during 2003.",
    "Any academic work available from the year 2000?",
    "Do you have research papers from 2001?",
    "Show me publications that came out in 1998.",
    "Looking for papers released in 1993.",
    "Find me studies from 1994.",
    "I’d like to view research from 1997.",
    "Can I see papers published back in 1996?",
    "Interested in publications from the year 1991.",
    "Could you bring up research articles from 1995?"
]

# Initialize a list to store precision for each query.
precision_scores = []

# Loop over each query to perform retrieval, display years, and calculate evaluation metrics.
for query in sample_queries:
    expected_year = extract_expected_year(query)
    print("Query:", query)
    print("Expected Year:", expected_year)

    # Project the query into the SVD space using the previously defined vectorizer, VT, and normalizer.
    query_vector = vectorizer.transform([query]).toarray()
    query_svd = query_vector.dot(VT.T)
    query_svd_normalized = normalizer.transform(query_svd)

    # Compute cosine similarities between the query SVD and all normalized document vectors.
    cosine_similarities = cosine_similarity(query_svd_normalized, U_normalized)

    # Retrieve the indices of the top 5 most similar documents.
    top_indices = np.argsort(cosine_similarities.flatten())[-5:][::-1]

    # Print out the years of the top 5 documents.
    print("Top 5 Document Years:")
    for idx in top_indices:
        print(df.iloc[idx]['year'])

    # Count the number of retrieved documents where the document year matches the expected year.
    TP = 0
    for idx in top_indices:
        doc_year = str(df.iloc[idx]['year']).strip()
        if doc_year == expected_year:
            TP += 1

    # Calculate precision based on the 5 retrieved documents.
    precision = TP / 5.0
    precision_scores.append(precision)

    print("Precision: {:.2f}".format(precision))
    print("-" * 80)

# Calculate average precision over all queries.
average_precision = np.mean(precision_scores)
print("Average Precision over all queries: {:.2f}".format(average_precision))


Query: I'm looking for research papers from 2016.
Expected Year: 2016
Top 5 Document Years:
2006
2013
2007
2016
2016
Precision: 0.40
--------------------------------------------------------------------------------
Query: Can you show me studies published in 2008?
Expected Year: 2008
Top 5 Document Years:
2002
2008
2016
2008
2016
Precision: 0.40
--------------------------------------------------------------------------------
Query: I’d like to see academic publications from 2009.
Expected Year: 2009
Top 5 Document Years:
2009
2009
2008
2009
2016
Precision: 0.60
--------------------------------------------------------------------------------
Query: Do you have any papers from 2015?
Expected Year: 2015
Top 5 Document Years:
2016
2008
2015
2015
2015
Precision: 0.60
--------------------------------------------------------------------------------
Query: Please find research papers that came out in 2006.
Expected Year: 2006
Top 5 Document Years:
2006
2013
2007
2017
2015
Precision: 0.20
------

In [ ]:
#Titles test

import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# List of research paper titles to be used as search query prompts.
titles = [
    "Teaching reconfigurable systems: methods, tools, tutorials, and projects",
    "A Novel Rekeying Message Authentication Procedure Based on Winternitz OTS and Reconfigurable Hardware Architectures",
    "The Complementarity of Information Technology Infrastructure and E-Commerce Capability: A Resource-Based Assessment of Their Business Value",
    "The Monty Hyams Archive: A new resource for the information history of the late twentieth century",
    "Dynamic signaling and market breakdown",
    "Enhanced lbs discovery in a decentralized registry based web services environment",
    "AGE AND VISUAL IMPAIRMENT DECREASE DRIVING PERFORMANCE AS MEASURED ON A CLOSED-ROAD CIRCUIT",
    "Conservative ambiguity detection in context-free grammars",
    "Design and implementation of a predictive control system for a photovoltaic generator",
    "Protocol Interference Between UP-and Downlink Channels in HSDPA",
    "On protocol and physical interference models in Poisson wireless networks",
    "T-S fuzzy model identification and the fuzzy model based controller design",
    "Secure and Flexible Trace-Based Debugging of Systems-on-Chip",
    "Performance analysis of the full-duplex enabled decode-and-forward two-way relay system",
    "Automated adaptive support for task and information prioritizing",
    "A Near-optimal Solution for the Heterogeneous Multi-processor Single-level Voltage Setup Problem",
    "Multiway pruning for efficient iceberg cubing",
    "Quality Credit Evaluation based on TOPSIS: Evidence from Air-conditioning Market in China",
    "Corrigendum: S4 enriched multimodal categorial grammars are context-free: Corrigendum",
    "Ms Pac-Man versus Ghost Team CEC 2011 competition",
    "Double-sampled cascaded sigma-delta modulator topologies for low oversampling ratios",
    "Learning spectral graph transformations for link prediction",
    "Generating Optimal Face Image in Face Recognition System",
    "Visual Correlates of Fixation Selection: A Look at the Spatial Frequency Domain",
    "A new crank arm based load cell, with built-in conditioning circuit and strain gages, to measure the components of the force applied by a cyclist"
]

# Initialize a list to store precision values for each query.
precision_scores = []

# Loop through each title query.
for query in titles:
    # Set the expected title as the cleaned (lowercase, stripped) version of the query.
    expected_title = query.strip().lower()
    print("Query:", query)
    print("Expected Title:", expected_title)

    # Project the query into the SVD space.
    # (Ensure that the variables 'vectorizer', 'VT', and 'normalizer' are defined in your environment)
    query_vector = vectorizer.transform([query]).toarray()
    query_svd = query_vector.dot(VT.T)
    query_svd_normalized = normalizer.transform(query_svd)

    # Compute cosine similarities between the query vector and all normalized document vectors.
    cosine_similarities = cosine_similarity(query_svd_normalized, U_normalized)

    # Retrieve the indices of the top 5 most similar documents.
    top_indices = np.argsort(cosine_similarities.flatten())[-1:][::-1]

    # Display the cleaned titles of the top 5 documents.
    print("Top 5 Document Titles:")
    for idx in top_indices:
        doc_title = df.iloc[idx]['title'].lower().strip()
        doc_title = ' '.join(doc_title.split())  # Clean extra spaces.
        print(doc_title)

    # Count the number of retrieved documents whose title exactly matches the expected title.
    TP = 0
    for idx in top_indices:
        doc_title = df.iloc[idx]['title'].lower().strip()
        doc_title = ' '.join(doc_title.split())
        if doc_title == expected_title:
            TP += 1

    # Calculate the precision (number of exact matches divided by 5).
    precision = TP / 1
    precision_scores.append(precision)

    print("Precision: {:.2f}".format(precision))
    print("-" * 80)

# Calculate and display the average precision across all query titles.
average_precision = np.mean(precision_scores)
print("Average Precision over all queries: {:.2f}".format(average_precision))


Query: Teaching reconfigurable systems: methods, tools, tutorials, and projects
Expected Title: teaching reconfigurable systems: methods, tools, tutorials, and projects
Top 5 Document Titles:
teaching reconfigurable systems: methods, tools, tutorials, and projects
Precision: 1.00
--------------------------------------------------------------------------------
Query: A Novel Rekeying Message Authentication Procedure Based on Winternitz OTS and Reconfigurable Hardware Architectures
Expected Title: a novel rekeying message authentication procedure based on winternitz ots and reconfigurable hardware architectures
Top 5 Document Titles:
a novel rekeying message authentication procedure based on winternitz ots and reconfigurable hardware architectures
Precision: 1.00
--------------------------------------------------------------------------------
Query: The Complementarity of Information Technology Infrastructure and E-Commerce Capability: A Resource-Based Assessment of Their Business Value


In [ ]:
#authors test
import re
import numpy as np

# Function to extract author names from a query
def extract_expected_authors(query):
    """
    Extracts a list of author names from the query by matching capitalized names.
    Returns a cleaned list of author names in lowercase.
    """
    matches = re.findall(r'[A-Z][a-z]+(?:\s[A-Z]\.)?(?:\s[A-Z][a-z]+)+', query)
    authors = [author.strip().lower() for author in matches]
    return authors

# Example author-based prompts
sample_queries = [

    "Can you show me papers authored by Stéphane Bonneau, Maxime Dahan, and Laurent D. Cohen?",
    "I'm looking for publications from Michael J. Swain and Paul R. Cooper.",
    "Please find articles written by Udai Shanker, Manoj Misra, Anil K. Sarje, and Rahul Shisondia.",
    "Any research papers by Wentao Fan and Nizar Bouguila?",
    "Do you have any works authored by Till Jochen Kniffka and Horst Ecker?",
    "Show me publications from Pawel A. Dmochowski and Peter J. McLane.",
    "Retrieve research by Daniel Ramot, Menahem Friedman, Gideon Langholz, Ron Milo, and Abraham Kandel.",
    "I want to explore papers written by Xuan Zhang, Antonis Papachristodoulou, and Na Li.",
    "Please show research publications by Feixiong Luo, Guofeng Cao, Kevin Mulligan, and Xiang Li.",
    "Are there any articles from Olivier Parodi, Lionel Lapierre, and Bruno Jouvencel?",
    "Give me research papers authored by Yang Hong, Changcheng Huang, and James Yan.",
    "Looking for academic work from Carlos E. Pedreira, Leonardo Macrini, Marcelo Gerardin Poirot Land, and Elaine S. Costa.",
    "Find studies authored by Srinivas Bodapati and Farid N. Najm.",
    "I’m interested in papers by Georgia Tseliou, Konstantinos Samdanis, Ferran Adelantado, Xavier Costa Perez, and Christos V. Verikoukis.",
    "Show me articles published by Luo Xu, Ji Wu, and Chao Liu.",
    "Could you retrieve research written by Waleed Arafa and Ragia Ibrahim?",
    "I'd like to see works by Andy Hunt and Dave Thomas.",
    "Get me publications from Ang Yang, Hussein A. Abbass, and Ruhul A. Sarker.",
    "Any papers by Tan Huai, Paul Ho, and Sami Muhaidat?",
    "I’m searching for research authored by Sangwook Bae, Giyoung Nam, and KyoungSoo Park.",
    "Can I see publications from Sarah Alhumoud and Lewis M. Mackenzie?",
    "I'd like to find papers written by Douglas B. Lenat.",
    "Looking for research articles from Zhe Yang, Yuting Bao, Chuhao Luo, Xingya Zhao, Siyu Zhu, Chunyi Peng, Yunxin Liu, and Xinbing Wang.",
    "Retrieve papers authored by Ram D. Gopal and Ram Ramesh.",
    "Show me research by Julia Buwaya and José D. P. Rolim."
]

# Precision scores list
precision_scores = []

# Evaluate each query
for query in sample_queries:
    expected_authors = extract_expected_authors(query)
    print("Query:", query)
    print("Expected Authors:", expected_authors)

    # Project the query into the SVD space
    query_vector = vectorizer.transform([query]).toarray()
    query_svd = query_vector.dot(VT.T)
    query_svd_normalized = normalizer.transform(query_svd)

    # Compute cosine similarity
    cosine_similarities = cosine_similarity(query_svd_normalized, U_normalized)
    top_indices = np.argsort(cosine_similarities.flatten())[-5:][::-1]

    print("Top 5 Document Authors:")
    TP = 0
    for idx in top_indices:
        doc_authors_raw = df.iloc[idx]['authors']

        # Normalize the authors from the dataset
        if isinstance(doc_authors_raw, list):
            doc_authors = [a.lower().strip() for a in doc_authors_raw]
        else:
            doc_authors = [a.strip().lower() for a in re.split(r',|;', str(doc_authors_raw)) if a.strip()]

        print(doc_authors)

        # Count as TP if all expected authors are in the document authors
        if all(exp_author in doc_authors for exp_author in expected_authors):
            TP += 1

    # Compute precision
    precision = TP / 5.0
    precision_scores.append(precision)
    print("Precision: {:.2f}".format(precision))
    print("-" * 80)

# Average precision
average_precision = np.mean(precision_scores)
print("Average Precision over all queries: {:.2f}".format(average_precision))


Query: Can you show me papers authored by Stéphane Bonneau, Maxime Dahan, and Laurent D. Cohen?
Expected Authors: ['maxime dahan', 'laurent d. cohen']
Top 5 Document Authors:
["['ajay k. gupta'", "'jagadish venkataraman'", "'oliver m. collins']"]
["['xiaojing huang'", "'yunxin li']"]
["['alexander neergaard olesen'", "'julie anja engelhard christensen'", "'helge bjarup dissing sørensen'", "'poul jennum']"]
["['aleda v. roth']"]
["['stathis delivorias'", "'sebastian rudolph']"]
Precision: 0.00
--------------------------------------------------------------------------------
Query: I'm looking for publications from Michael J. Swain and Paul R. Cooper.
Expected Authors: ['michael j. swain', 'paul r. cooper']
Top 5 Document Authors:
["['jean-raymond abrial'", "'dominique cansell']"]
["['prosenjit bose'", "'hua guo'", "'evangelos kranakis'", "'anil maheshwari'", "'pat morin'", "'jason morrison'", "'michiel h. m. smid'", "'yihui tang']"]
["['rajni nigam'", "'diane h. munzenmaier'", "'elizabet